In [1]:
import sqlalchemy
import pandas as pd

In [2]:
query='''
SELECT tl.[TransactionId], p.[StockCode],p.[Description], c.[Category],th.[RecordURL],c.[Name], tl.[Quantity],
tl.[ImpactAmount], th.[SiteName], th.[Type], p.[Class]
  FROM [PowerBI].[dbo].[TransactionLine] tl
  LEFT JOIN [PowerBI].[dbo].[Subsidiary] s ON tl.SubsidiaryId = s.[SubsidiaryId] 
  LEFT JOIN [PowerBI].[dbo].[Product] p ON tl.[ProductId] =p.[ProductId]
  LEFT JOIN [PowerBI].[dbo].[TransactionHeader] th ON tl.[TransactionId]=th.[TransactionId]
  LEFT JOIN [PowerBI].[dbo].[Customer] c ON c.[CustomerId]= th.[Entity]

  Where TransactionType In ('Sales Order') AND s.[Name]='Wilton Bradley Limited'
  AND c.[Category] ='B2C' AND c.[Name] NOT IN ('WB Amazon UK Reversal Reimbursements','Influencer Account')
  AND p.[StockCode] NOT IN ( 'CARRIAGE1','1PROM ','B2C Shipping Charge Item ','Cancelled Order Item')
  AND tl.[TransactionId]>0
 
ORDER BY tl.[TransactionId] Desc

'''

In [3]:
engine=sqlalchemy.create_engine("mssql://@192.168.35.102/PowerBi?driver=SQL Server")
connection=engine.connect()
result=connection.execute(sqlalchemy.text(query))

data=[row for row in result]
connection.close()

df=pd.DataFrame(data)
df.head()

,TransactionId,StockCode,Description,Category,RecordURL,Name,Quantity,ImpactAmount,SiteName,Type,Class
0,17994092,BW52560,Pong Champion Pool Game,B2C,https://6371792.app.netsuite.com/app/accountin...,Rachel unknown,1,19.9900,Boardmasters: Amazon (MFN),Sales Order,Bestway
1,17994091,FW352-06,Osp Osx Adult Wetboot - Black - Size 6 (39/40),B2C,https://6371792.app.netsuite.com/app/accountin...,Scott Morrell,1,9.9900,Boardmasters: Amazon (MFN),Sales Order,Osprey
2,17994090,TY6139,Playhouse Mucky Mud Kitchen,B2C,https://6371792.app.netsuite.com/app/accountin...,Kirsty Temple,2,67.4800,Boardmasters: eBay,Sales Order,Toyrific
3,17994089,SK5024-L,Osp Kids Pads - Flower - L (8-12 yrs),B2C,https://6371792.app.netsuite.com/app/accountin...,mrs meganlindsayevanstarrier,1,8.3300,Boardmasters: Amazon (MFN),Sales Order,Osprey
4,17994088,TY4605,Bingo Game,B2C,https://6371792.app.netsuite.com/app/accountin...,Amy Foster,1,8.3300,Boardmasters: Amazon (MFN),Sales Order,Toyrific


In [4]:
df["Class"].value_counts()

Class
Osprey              127908
Lay-Z-Spa            44548
Clearwater           31812
Bestway              26598
Toyrific             16478
ASS                   9277
Xootz                 8817
VW                    6020
Urban Beach           1229
No Product Class        90
Yello                    1
Name: count, dtype: int64

In [5]:
#To check the number of transactions which have more than 5 stockcodes
tran=df.groupby("TransactionId")['StockCode'].count()
tran5= tran[tran>5]
tran5.count()

1858

In [6]:
product_count=df.groupby("TransactionId")['StockCode'].transform('count')
df["Count"]=product_count
df1=df[df["Count"]>10]
df1.to_csv("S:/sales office/Vishnu/Market Basket analysis/Large Basket.csv")

In [7]:
df2=df[["TransactionId", "StockCode"]]
df2

,TransactionId,StockCode
0,17994092,BW52560
1,17994091,FW352-06
2,17994090,TY6139
3,17994089,SK5024-L
4,17994088,TY4605
...,...,...
272774,32077,P6575ASS16
272775,31976,OL0186
272776,31671,B-BW54123GASS17
272777,30241,BGG1324B-S


In [8]:
basket=(df2.groupby(["TransactionId", "StockCode"])["StockCode"].count().unstack().reset_index().fillna(0)).set_index('TransactionId')
basket2=basket.applymap(lambda x:1 if x>0 else 0)
basket2

StockCode,54112GASS15,54123GASS17,54148GASS16,54173GASS18,54174GASS18,54175GASS18,56223ASS19,56229ASS19,56241ASS19,56338ASS19,...,WST0060,WST0061,WST0062,WST0063,WST0064,WST0065,WST0066,WST0067,WST0068,bw58210
TransactionId,,,,,,,,,,,,,,,,,,,,,
30241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31671,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31976,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32077,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32178,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17994088,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17994089,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17994090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
total_transaction=len(basket2)
min_transactions=

min_support=min_transactions/total_transaction
min_support

1.9275343462526324e-05

In [13]:
from mlxtend.frequent_patterns import apriori, association_rules
frequent= apriori(basket2, min_support=0.01, use_colnames=True, low_memory=True)
frequent

C:\Users\v.murleedharan\AppData\Local\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.010409,(B-BW54123GASS17)
1,0.033717,(BW60311)
2,0.016196,(BW65350)
3,0.019531,(BW6714NGB-1)
4,0.016490,(BW6714PGB-1)
5,0.016056,(CH0017)
6,0.028147,(CH0018)
7,0.030344,(CH0019)
8,0.010534,(CH0043)
9,0.011050,(P05328)


In [381]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent= apriori(basket2, min_support=0.0024, use_colnames=True, low_memory=True)
rules=association_rules(frequent, metric='lift', min_threshold=1)
rules

C:\Users\v.murleedharan\AppData\Local\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(BW13222),(BW60311),0.004902,0.033770,0.002771,0.565217,16.737359,0.002605,2.222329,0.944886
1,(BW60311),(BW13222),0.033770,0.004902,0.002771,0.082054,16.737359,0.002605,1.084048,0.973115
2,(BW13222),(CH0018),0.004902,0.028209,0.002989,0.609684,21.613463,0.002851,2.489754,0.958431
3,(CH0018),(BW13222),0.028209,0.004902,0.002989,0.105959,21.613463,0.002851,1.113034,0.981417
4,(CH0019),(BW13222),0.030418,0.004902,0.002630,0.086479,17.639868,0.002481,1.089299,0.972904
...,...,...,...,...,...,...,...,...,...,...
151,"(CH0018, BW60163GB)","(CH0019, BW60311)",0.002853,0.024973,0.002853,1.000000,40.044035,0.002782,inf,0.977818
152,(CH0019),"(BW60311, CH0018, BW60163GB)",0.030418,0.002853,0.002853,0.093805,32.875776,0.002767,1.100366,1.000000
153,(BW60311),"(CH0019, CH0018, BW60163GB)",0.033770,0.002853,0.002853,0.084493,29.612251,0.002757,1.089174,1.000000
154,(CH0018),"(CH0019, BW60311, BW60163GB)",0.028209,0.002853,0.002853,0.101151,35.450283,0.002773,1.109359,1.000000


In [382]:
frequent

,support,itemsets
0,0.009844,(B-BW54112GASS15)
1,0.010464,(B-BW54123GASS17)
2,0.003154,(B-BW54173GASS18)
3,0.004578,(B-BW54175GASS18)
4,0.002485,(B-BWF4C009B-02AS)
...,...,...
153,0.002427,"(CH0019, CH0046, BW60311)"
154,0.003003,"(CH0034, CH0035, CH0033)"
155,0.002538,"(CH0019, BW13222, BW60311, CH0018)"
156,0.003454,"(CH0019, BW60311, BW60047GB, CH0018)"


In [383]:
rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(x))
rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(x))
rules.to_csv("S:/sales office/Vishnu/Market Basket analysis/rules.csv", index=False)

In [248]:
selected=basket2[(basket2["BW60047GB"]==1) & (basket2["CH0019"]==1)].index.tolist()
selected

[389633,
 1915045,
 2006175,
 2057269,
 2061089,
 2137756,
 2389708,
 2393980,
 2480753,
 2550372,
 2556385,
 2612320,
 2671264,
 2727364,
 2741863,
 2802865,
 2848239,
 3144777,
 3182783,
 3232668,
 3232669,
 3257337,
 3289118,
 3304668,
 3320485,
 3330551,
 3330552,
 3332727,
 3337678,
 3337883,
 3348657,
 3348991,
 3387008,
 3387131,
 3387494,
 3391598,
 3405630,
 3413651,
 3481230,
 3483841,
 3546367,
 3571904,
 3591638,
 3592610,
 3592611,
 3601870,
 3635224,
 3639214,
 3641745,
 3645868,
 3650323,
 3650324,
 3652781,
 3662189,
 3668354,
 3672854,
 3673476,
 3683914,
 3685841,
 3686687,
 3706238,
 3754772,
 3754774,
 3759388,
 3762284,
 3804143,
 3831312,
 3883227,
 3902348,
 3910348,
 3937290,
 3945761,
 3954881,
 3963884,
 3976650,
 4005068,
 4013986,
 4028644,
 4043174,
 4045082,
 4045832,
 4050145,
 4056039,
 4060581,
 4062274,
 4064626,
 4066465,
 4068183,
 4087840,
 4099444,
 4099556,
 4100517,
 4102051,
 4102377,
 4105274,
 4106840,
 4108263,
 4133349,
 4143288,
 4145176,
 

In [ ]:
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequent_len'] = rules['consequents'].apply(lambda x: len(x))
filtered_rules = rules[(rules['antecedent_len'] == 1) & (rules['consequent_len'] == 1)]


In [196]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,CH0017,BW56408GB-21,0.016175,0.005291,0.004884,0.301921,57.059424,0.004798,1.424922,0.998627
1,BW56408GB-21,CH0017,0.005291,0.016175,0.004884,0.922936,57.059424,0.004798,12.766301,0.987701
2,BW60311,BW60047GB,0.033748,0.004568,0.003495,0.103567,22.672218,0.003341,1.110437,0.989279
3,BW60047GB,BW60311,0.004568,0.033748,0.003495,0.765143,22.672218,0.003341,4.114222,0.960280
4,BW60047GB,CH0018,0.004568,0.028209,0.003534,0.773645,27.425325,0.003405,4.293217,0.967959
...,...,...,...,...,...,...,...,...,...,...
59,"BW60047GB, CH0018","CH0019, BW60311",0.003534,0.024966,0.003456,0.978022,39.173555,0.003368,44.364030,0.977929
60,CH0019,"BW60311, BW60047GB, CH0018",0.030423,0.003481,0.003456,0.113611,32.640894,0.003350,1.124246,0.999780
61,BW60311,"CH0019, BW60047GB, CH0018",0.033748,0.003456,0.003456,0.102417,29.631329,0.003340,1.110252,1.000000
62,BW60047GB,"CH0019, BW60311, CH0018",0.004568,0.024165,0.003456,0.756642,31.310959,0.003346,4.009871,0.972505


# Only Osprey

In [260]:
#testing
df_osprey=df[df["Class"]=="Osprey"]
basket_osprey=(df_osprey.groupby(["TransactionId", "StockCode"])["StockCode"].count().unstack().reset_index().fillna(0)).set_index('TransactionId')
basket_osprey=basket_osprey.applymap(lambda x:1 if x>0 else 0)
basket_osprey

StockCode,BGG1324B-L,BGG1324B-M,BGG1324B-S,BGG1324P-L,BGG1324P-M,BGG1324P-S,BGG1372-BL,BGG1372-RD,BGG1373-BL,BGG1373-RD,...,WST0042,WST0060,WST0061,WST0062,WST0063,WST0064,WST0065,WST0066,WST0067,WST0068
TransactionId,,,,,,,,,,,,,,,,,,,,,
30241,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32178,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43540,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43743,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17702932,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17704770,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17704771,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [285]:
total_transaction=len(basket_osprey)
min_transactions=50

min_support=min_transactions/total_transaction
min_support

0.0004526484460578847

In [286]:
frequent_osprey=apriori(basket_osprey, min_support=0.0004, use_colnames=True, low_memory=True)
rules_osprey=association_rules(frequent_osprey, metric='lift', min_threshold=1)
rules_osprey


C:\Users\v.murleedharan\AppData\Local\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(BGG1401),(BGG1403),0.002426,0.002942,0.000471,0.194030,65.946866,0.000464,1.237090,0.987231
1,(BGG1403),(BGG1401),0.002942,0.002426,0.000471,0.160000,65.946866,0.000464,1.187588,0.987742
2,(BGG1500),(BGG1749),0.007324,0.007487,0.000489,0.066749,8.915561,0.000434,1.063501,0.894387
3,(BGG1749),(BGG1500),0.007487,0.007324,0.000489,0.065296,8.915561,0.000434,1.062022,0.894534
4,(BGG1501),(BGG1502),0.008193,0.007351,0.000407,0.049724,6.764207,0.000347,1.044590,0.859202
5,(BGG1502),(BGG1501),0.007351,0.008193,0.000407,0.055419,6.764207,0.000347,1.049997,0.858474
6,(BGG1750),(BGG1502),0.007786,0.007351,0.000416,0.053488,7.276329,0.000359,1.048745,0.869336
7,(BGG1502),(BGG1750),0.007351,0.007786,0.000416,0.056650,7.276329,0.000359,1.051799,0.868956
8,(BGG1751),(BGG1502),0.004898,0.007351,0.000471,0.096118,13.075522,0.000435,1.098207,0.928067
9,(BGG1502),(BGG1751),0.007351,0.004898,0.000471,0.064039,13.075522,0.000435,1.063188,0.930360


In [278]:
selected=basket_osprey[(basket_osprey["OSG0057-BL"]==1) & (basket_osprey["OSG0057-BK"]==1)].index.tolist()
len(selected)

50

In [250]:
df_osprey=df[df["Class"]=="Osprey"]
df_osprey["SiteName"].value_counts()

SiteName
Boardmasters: Amazon (MFN)    54807
WB: Amazon FBA UK (AFN)       32339
Osprey Surf                   31904
Boardmasters: eBay             5561
Bertie: Osprey                 1238
Boardmasters: Decathlon         966
Hydroforce                      286
Bertie: Decathlon                 5
Bertie: Xootz                     3
Bertie: Hydroforce                1
WB: Amazon FBA ES (AFN)           1
Name: count, dtype: int64

In [253]:
df_lay=df[df["Class"]=="Lay-Z-Spa"]
df_lay["SiteName"].value_counts()

SiteName
LayZSpa                       40499
Bestway Store                  2814
Bertie: Lay-z-spa               496
Boardmasters: Amazon (MFN)      399
WB: Amazon FBA UK (AFN)          33
Bertie: Bestway Stores           26
Hydroforce                       18
Boardmasters: Decathlon           3
Marketing/Influencer              3
Boardmasters: eBay                2
Bertie: Osprey                    1
Name: count, dtype: int64

In [255]:
df["SiteName"].value_counts()

SiteName
Boardmasters: Amazon (MFN)    82898
LayZSpa                       68676
WB: Amazon FBA UK (AFN)       43300
Osprey Surf                   32442
Bestway Store                 20941
Bertie: Bestway Support        9205
Boardmasters: eBay             6927
Xootz                          2258
Bertie: Osprey                 1451
Boardmasters: Decathlon         971
Hydroforce                      812
Bertie: Lay-z-spa               797
Bertie: Bestway Stores          383
Bertie: Xootz                    56
WB: Amazon FBA IT (AFN)          18
Bertie: Urban Beach              10
Bertie: Hydroforce                6
Bertie: Decathlon                 5
Marketing/Influencer              4
Bertie: JSF                       2
WB: Amazon FBA ES (AFN)           2
WB: Amazon FBA FR (AFN)           1
Bertie: Clearwater                1
Name: count, dtype: int64

# Only Boardmasters: Amazon (MFN) 

In [292]:
df_mfn=df[df["SiteName"]=="LayZSpa"]
basket_mfn=(df_mfn.groupby(["TransactionId","StockCode"])["StockCode"].count().unstack().reset_index().fillna(0)).set_index('TransactionId')
basket_mfn=basket_mfn.applymap(lambda x:1 if x>0 else 0)
basket_mfn

StockCode,B-BW54112GASS15,B-BW54123GASS17,B-BW54173GASS18,B-BW54174GASS18,B-BW54175GASS18,B-BWF4C009B-02AS,B-BWF4C009B-02ASS17,B-BWF6A526-02ASS,B-BWF6H189ASS16,B-BWP00217,...,CH0041,CH0043,CH0045,CH0046,CH0057,CH0058,CH0060,CH1000,CH1001,bw58210
TransactionId,,,,,,,,,,,,,,,,,,,,,
31671,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32686,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
57476,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
57987,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57988,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17683366,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17685153,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17685468,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [297]:
total_transaction=len(basket_mfn)
min_transactions=500

min_support=min_transactions/total_transaction
min_support

0.016135278172195688

In [298]:
frequent_mfn=apriori(basket_mfn, min_support=0.016, use_colnames=True, low_memory=True)
rules_mfn=association_rules(frequent_mfn, metric='lift', min_threshold=1)
rules_mfn

C:\Users\v.murleedharan\AppData\Local\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(BW13222),(BW60311),0.031722,0.219762,0.018265,0.575788,2.620049,0.011294,1.839265,0.638585
1,(BW60311),(BW13222),0.219762,0.031722,0.018265,0.083113,2.620049,0.011294,1.056050,0.792486
2,(BW13222),(CH0018),0.031722,0.184878,0.019814,0.624619,3.378544,0.013949,2.171450,0.727079
3,(CH0018),(BW13222),0.184878,0.031722,0.019814,0.107174,3.378544,0.013949,1.084509,0.863692
4,(CH0019),(BW13222),0.194430,0.031722,0.017458,0.089793,2.830611,0.011291,1.063799,0.802810
...,...,...,...,...,...,...,...,...,...,...
149,"(CH0018, BW60163GB)","(CH0019, BW60311)",0.019007,0.164870,0.019007,1.000000,6.065375,0.015874,inf,0.851311
150,(CH0019),"(BW60311, CH0018, BW60163GB)",0.194430,0.019007,0.019007,0.097759,5.143237,0.015312,1.087285,1.000000
151,(BW60311),"(CH0019, CH0018, BW60163GB)",0.219762,0.019007,0.019007,0.086490,4.550367,0.014830,1.073872,1.000000
152,(CH0018),"(CH0019, BW60311, BW60163GB)",0.184878,0.019007,0.019007,0.102810,5.408972,0.015493,1.093406,1.000000


# Only Domestic 

In [366]:
query2='''
SELECT tl.[TransactionId], p.[StockCode],p.[Description], c.[Category],th.[RecordURL],c.[Name], tl.[Quantity],
tl.[ImpactAmount], th.[SiteName], th.[Type], p.[Class]
  FROM [PowerBI].[dbo].[TransactionLine] tl
  LEFT JOIN [PowerBI].[dbo].[Subsidiary] s ON tl.SubsidiaryId = s.[SubsidiaryId] 
  LEFT JOIN [PowerBI].[dbo].[Product] p ON tl.[ProductId] =p.[ProductId]
  LEFT JOIN [PowerBI].[dbo].[TransactionHeader] th ON tl.[TransactionId]=th.[TransactionId]
  LEFT JOIN [PowerBI].[dbo].[Customer] c ON c.[CustomerId]= th.[Entity]

  Where TransactionType In ('Sales Order') AND s.[Name]='Wilton Bradley Limited' AND p.[StockCode] IS NOT NULL 
  AND c.[Category] ='Domestic' 
 

ORDER BY tl.[TransactionId] Desc
'''

In [367]:
engine=sqlalchemy.create_engine("mssql://@192.168.35.102/PowerBi?driver=SQL Server")
connection=engine.connect()
result=connection.execute(sqlalchemy.text(query2))

data=[row for row in result]
connection.close()

df_domestic=pd.DataFrame(data)
df_domestic.head()

,TransactionId,StockCode,Description,Category,RecordURL,Name,Quantity,ImpactAmount,SiteName,Type,Class
0,17758178,BU1259,Yel 28cm Crabbing Bucket Extra Large,Domestic,https://6371792.app.netsuite.com/app/accountin...,Pier Crabbing,48.0,45.6000,None,Sales Order,Yello
1,17758178,BGG1601,Yel Crab Drop Net With Rings And Handle,Domestic,https://6371792.app.netsuite.com/app/accountin...,Pier Crabbing,96.0,116.1600,None,Sales Order,Yello
2,17756825,BU1307,"Yel Recycled PP Spade - 14""",Domestic,https://6371792.app.netsuite.com/app/accountin...,Collaborate & Innovate LTS T/A Cosy,144.0,177.1200,None,Sales Order,Yello
3,17756825,BU1306,Yel Recycled PP Bucket - Single,Domestic,https://6371792.app.netsuite.com/app/accountin...,Collaborate & Innovate LTS T/A Cosy,72.0,112.3200,None,Sales Order,Yello
4,17756015,B219,23cm Giant Tennis Ball,Domestic,https://6371792.app.netsuite.com/app/accountin...,Test Customer,60.0,151.2000,None,Sales Order,Toyrific


In [368]:
df2_domestic.groupby("TransactionId")["StockCode"].count().sort_values(ascending= False)

TransactionId
23151       173
5793549     170
13529027    165
11037630    156
14654797    144
           ... 
7325279       1
7325025       1
7289872       1
7283405       1
17340         1
Name: StockCode, Length: 11380, dtype: int64

In [369]:
df2_domestic=df[["TransactionId", "StockCode"]]
df2_domestic

,TransactionId,StockCode
0,17748688,FWR1121-05
1,17748688,BU1308
2,17748688,FWR1126-10
3,17748688,FWR1126-11
4,17748688,FWR1128-07
...,...,...
65454,17341,WS1233
65455,17341,FWR1121-08
65456,17341,BW62030
65457,17341,WB022


In [370]:
basket_domestic=(df2_domestic.groupby(["TransactionId", "StockCode"])["StockCode"].count().unstack().reset_index().fillna(0)).set_index('TransactionId')
basket2_domestic=basket_domestic.applymap(lambda x:1 if x>0 else 0)
basket2_domestic

StockCode,A-BW21002-23,A-BW21048-23,A-BW21077-23,A-BW21080-23,A-BW24018-23,A-BW24023-23,A-BW24025,A-BW24029,A-BW24058-23,A-BW24060-23,...,WST0042,WST0062,WST0063,WST0064,WST0065,WST0066,WST0067,WST0068,bw58210,bw67225-23
TransactionId,,,,,,,,,,,,,,,,,,,,,
17340,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17341,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18652,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18657,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18663,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17711573,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17711699,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17713200,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [371]:
total_transaction=len(basket2_domestic)
min_transactions=100

min_support=min_transactions/total_transaction
min_support

0.008787346221441126

In [372]:
frequent_domestic= apriori(basket2_domestic, min_support=0.0087, use_colnames=True, low_memory=True)
rules_domestic=association_rules(frequent_domestic, metric='lift', min_threshold=1)
rules_domestic

C:\Users\v.murleedharan\AppData\Local\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(B318),(B317),0.020123,0.021968,0.013005,0.646288,29.419039,0.012563,2.765052,0.985847
1,(B317),(B318),0.021968,0.020123,0.013005,0.592000,29.419039,0.012563,2.401659,0.987707
2,(BGG1535),(BG930),0.025571,0.027856,0.009227,0.360825,12.953267,0.008514,1.520935,0.947016
3,(BG930),(BGG1535),0.027856,0.025571,0.009227,0.331230,12.953267,0.008514,1.457047,0.949241
4,(BG930),(BGG1553),0.027856,0.031283,0.009490,0.340694,10.890724,0.008619,1.469298,0.934202
...,...,...,...,...,...,...,...,...,...,...
116559,(FWR1126-10),"(FWR1126-08, FWR1126-11, FWR1124-5X, FWR1126-0...",0.018717,0.009139,0.008787,0.469484,51.372337,0.008616,1.867729,0.999237
116560,(FWR1126-07),"(FWR1126-08, FWR1126-11, FWR1124-5X, FWR1126-1...",0.018102,0.008963,0.008787,0.485437,54.159528,0.008625,1.925977,0.999631
116561,(FWR1124-4X),"(FWR1126-08, FWR1126-11, FWR1124-5X, FWR1126-1...",0.019684,0.009315,0.008787,0.446429,47.927898,0.008604,1.789625,0.998795
116562,(FWR1126-09),"(FWR1126-08, FWR1126-11, FWR1124-5X, FWR1126-1...",0.019508,0.009051,0.008787,0.450450,49.768215,0.008611,1.803202,0.999403


In [365]:
rules_domestic['antecedents'] = rules_domestic['antecedents'].apply(lambda x: ', '.join(x))
rules_domestic['consequents'] = rules_domestic['consequents'].apply(lambda x: ', '.join(x))
rules_domestic.to_csv("S:/sales office/Vishnu/Market Basket analysis/rules_domestic.csv", index=False)